In [9]:
#Ignore scikit learn deprication warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [2]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re


/Users/andrew/workspace/endangeredanimals/learning


/Users/andrew/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from numpy import genfromtxt
# train = pd.read_csv('train_reduced.csv', encoding="ISO-8859-1", index_col='_id')
# test = pd.read_csv('test_reduced.csv', encoding="ISO-8859-1", index_col='_id')
from numpy import genfromtxt
#myData = genfromtxt('train.csv',delimiter=',')

#train = genfromtxt('train_reduced_CME_test.csv',delimiter=',')
#test = genfromtxt('test_reduced_CME_test.csv',delimiter=',')

train = genfromtxt('train_reduced.csv',delimiter=',')
test = genfromtxt('test_reduced.csv',delimiter=',')

print(train.shape)
# test = test.dropna()
# test.target
train = train[~np.isnan(train).any(axis=1)]

y = train[:,4] # label
X = train[:,0:3] # data

(281, 5)


In [4]:
# from sklearn.cross_validation import cross_val_score

# # TODO use F-1 for scoring, or a hybrid if possible
# # we can use a for loop ['f1', 'accuracy', ]
# # http://scikit-learn.org/stable/modules/model_evaluation.html

# # logistic regression
# from sklearn.linear_model import LogisticRegression
# logreg = LogisticRegression()
# logreg.fit(X,y)
# print('Logistic Regression:')
# print(, cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

# # decision tree
# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier()
# clf.fit(X,y)
# print('Decision Tree:', cross_val_score(clf, X, y, cv=10, scoring='accuracy').mean())

# # random forest
# from sklearn.ensemble import RandomForestClassifier
# forest = RandomForestClassifier(n_estimators=10, max_depth=10)
# forest.fit(X,y)
# print('Random Forest:', cross_val_score(forest, X, y, cv=10, scoring='accuracy').mean())

# # support vector machine
# from sklearn.svm import LinearSVC
# svm = LinearSVC()
# svm.fit(X,y)
# print('SVM:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# # naive bayes
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# gnb.fit(X,y)
# print('Naive Bayes:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# # KNN
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=20)
# knn.fit(X,y)
# print('KNN:', cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean())


In [12]:
# Awesome sauce.  Now let's fill in missing values rather than dropping them
# reload train
train = genfromtxt('train_reduced_CME_test.csv',delimiter=',')
# print(train.shape)

# get rid of first row (I think this is just column labels)
train = train[1:,:]

# replace missing featureVals with 0.5 (halfway in between)
where_are_NaNs = np.isnan(train)
train[where_are_NaNs] = -1

y = train[:,4] # label
X = train[:,0:3] # data

In [6]:
# from sklearn.cross_validation import cross_val_score

# #TODO use F-1 for scoring, or a hybrid if possible

# # logistic regression
# from sklearn.linear_model import LogisticRegression
# logregFillNA = LogisticRegression()
# logregFillNA.fit(X,y)
# print('Logistic Regression:', cross_val_score(logregFillNA, X, y, cv=10, scoring='accuracy').mean())

# # decision tree
# from sklearn.tree import DecisionTreeClassifier
# dtFillNA = DecisionTreeClassifier()
# dtFillNA.fit(X,y)
# print('Decision Tree:', cross_val_score(dtFillNA, X, y, cv=10, scoring='accuracy').mean())

# # random forest
# from sklearn.ensemble import RandomForestClassifier
# forestFillNA = RandomForestClassifier(n_estimators=10, max_depth=10)
# forestFillNA.fit(X,y)
# print('Random Forest:', cross_val_score(forestFillNA, X, y, cv=10, scoring='accuracy').mean())

# # support vector machine
# from sklearn.svm import LinearSVC
# svmFillNA = LinearSVC()
# svmFillNA.fit(X,y)
# print('SVM:', cross_val_score(svmFillNA, X, y, cv=10, scoring='accuracy').mean())

# # naive bayes
# from sklearn.naive_bayes import GaussianNB
# gnbFillNA = GaussianNB()
# gnbFillNA.fit(X,y)
# print('Naive Bayes:', cross_val_score(gnbFillNA, X, y, cv=10, scoring='accuracy').mean())

# # KNN
# from sklearn.neighbors import KNeighborsClassifier
# knnFillNA = KNeighborsClassifier(n_neighbors=20)
# knnFillNA.fit(X,y)
# print('KNN:', cross_val_score(knnFillNA, X, y, cv=10, scoring='accuracy').mean())


In [11]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

models = [LogisticRegression(), 
          DecisionTreeClassifier(), 
          RandomForestClassifier(n_estimators=10, max_depth=10),
          LinearSVC(),
          GaussianNB(),
          KNeighborsClassifier(n_neighbors=20)
         ]
names =  ['Logistic Regression', 
          'Decision Tree',
          'Random Forest',
          'SVM',
          'Naive Bayes',
          'KNN']

metrics = ['accuracy', 'precision', 'recall', 'f1']

def compute_metrics(name, model):
    model.fit(X,y)
    print("{}:".format(name))
    for metric in metrics:
        print(metric + ':', cross_val_score(model, X, y, cv=10, scoring=metric).mean())
    print('')

for i in range(len(names)):
    compute_metrics(names[i], models[i])

Logistic Regression:
accuracy: 0.868144499179
precision: 0.0
recall: 0.0
f1: 0.0

Decision Tree:
accuracy: 0.885905856596
precision: 0.641666666667
recall: 0.441666666667
f1: 0.516428571429

Random Forest:
accuracy: 0.896506112023
precision: 0.766666666667
recall: 0.441666666667
f1: 0.509047619048

SVM:
accuracy: 0.646524356869
precision: 0.0249042145594
recall: 0.5
f1: 0.0

Naive Bayes:
accuracy: 0.868144499179
precision: 0.0
recall: 0.0
f1: 0.0

KNN:
accuracy: 0.868144499179
precision: 0.0
recall: 0.0
f1: 0.0

